In [1]:
// подключаем необходимые библиотеки
import $ivy.`org.scalanlp::breeze:1.0`
import scala.collection.mutable.ArrayBuffer
import breeze.linalg._

import $ivy.$                         

import scala.collection.mutable.ArrayBuffer

import breeze.linalg._

In [ ]:
// читаем файл с данными
val dataFile = io.Source.fromFile("Dataset.csv")

// подготавливаем матрицу
var dataMatrix = DenseMatrix.zeros[Double](0,28)

var i = 0
// Пробегаемся по каждай строке файла
for (line <- dataFile.getLines.drop(1)) { 
    
    // готовим массив для дальнейшего его перегона в матрицу 
    var dataArray = Array[Double]()
    
    // делим строку на элементы и обрезаем пробелы
    for (elem <- line.split(",")map(_.trim)){
        // если значения нет, то добавляем 0
        if(elem == "") {
            dataArray = dataArray :+ 0.0 
        // иначе добавляем в массив значение елемента
        }else{
            dataArray = dataArray :+ elem.toDouble
        }
    }
    
    // наполняем матрицу
    dataMatrix = DenseMatrix.vertcat(dataMatrix, DenseMatrix(DenseVector(dataArray)))
    i += 1 // без счетчика у меня напроч умирал юпитер, видимо потому, что пытался вывести все значения матрицы 
}



In [ ]:
// разбиваем данные
val X_all = dataMatrix(0 to dataMatrix.rows - 1, 0 to dataMatrix.cols-2)
val Y_all = dataMatrix(::,27)

### Посмотрим на корреляцию в данных (не осилил это сделать в скале, сделал через LibreOffice)

##### для удобства разбил на две матрицы корреляции, чтобы нормально влезло в экран

![title](dataset_corr.png)


Как мы можем заметить, самая высока степерь корреляции с целевым признаком output у признака **comm24**. Далее по списку идут: **diff2448 , comm24_1 , commBase**. Их и будем использовать в качестве описываемых признаков.



In [ ]:
// вырезаем необходимые признаки (4,5,7,8) и собираем новую матрцу

val X_commBase = DenseMatrix(X_all(::,4))
val X_comm24 = DenseMatrix(X_all(::,5))
val X_comm24_1 = DenseMatrix(X_all(::,7))
val X_diff2448 = DenseMatrix(X_all(::,8))
var X = DenseMatrix.zeros[Double](X_commBase(::,0).length,0)

X = DenseMatrix.horzcat(X_commBase.t, X_comm24.t)
X = DenseMatrix.horzcat(X, X_comm24_1.t)
X = DenseMatrix.horzcat(X, X_diff2448.t)

In [ ]:
// Проверяем, что размерности не поехали
X.rows
X.cols

In [ ]:
// разделим выборки на train и test
val test_count = (X.rows * 0.67).toInt
val X_train = X(0 to test_count, 0 to X.cols - 1)
val X_test = X(test_count to X.rows - 1, 0 to X.cols - 1)
val Y_train = Y_all(0 to test_count)
val Y_test = Y_all(test_count to Y_all.length - 1)
assert(X_train.rows == Y_train.length)
assert(X_test.rows == Y_test.length)

In [ ]:
// применяем регрессионные модели на 4х колонках
import breeze.stats.regression.{leastSquares, lasso}
val squaresModel = leastSquares(X_train, Y_train)
val lassoModel = lasso(X_train, Y_train, 0.01)


In [ ]:
// применим наши модели на тестовой выборке

assert(X_test.cols == squaresModel.coefficients.length)
val Y_pred_squaresModel = X_test * squaresModel.coefficients
val Y_pred_lassoModel = X_test * lassoModel.coefficients


### В качестве функции потерь будем использовать MAE т.к. нам необходимо понимать, на сколько комментариев мы промахиваемся в ту или иную сторону

In [ ]:
import math.abs
val mae = (X: DenseVector[Double], X_pred: DenseVector[Double]) => {
    ((X - X_pred).map(x => abs(x)).sum) / X.length
}

val mae_test_squaresModel = mae(Y_test, Y_pred_squaresModel)
val mae_test_lassoModel = mae(Y_test, Y_pred_lassoModel)

##### Судя по результатам оценки, lassoModel показывает немного большую точность чем squaresModel на текущих данных

## Вычислим коэфиценты своими силами и сравним результаты

### Т.к. данных у нас не так много, можно использовать самую простую реализацию

In [ ]:
import breeze.linalg.inv
val X_train_t = X_train.t
val coef_custom = inv((X_train_t * X_train)) * X_train_t * Y_train
var Y_pred_custom = X_test * coef_custom
val mae_test_custom = mae(Y_test, Y_pred_custom)

### Итоговые значения средних абсолютных ошибок: 

* lassoModel = 5.984693119061619
* squaresModel = 6.231902929890508
* custom = 6.231902929890598

Кастомная реализация показала практически тот же результат, что и squaresModel. В целом не плохо. 